# Recommender System 

- for more details about recommender Systems look for the book Jannach and Zanker
- Netflix recammendation example if you want to try using SVD , SVD ++
- the two most common types of recommender system are 
    - Content-Based
         - produces recommendation based on the usesr's attitude to items, that is it uses the wisdom of the crowd to recommend items 
    - Collaborative Filtering  Cf
        - focus on the attributes if the items and give you recommendations based on the similarity between them 
     
- in general Collaborative filtering CF is more commonly used than Content Based because it gives better result and easy to understand 

- theses technique aim to fill the missing value of the user item assoction matrix in way to predict the rate of the movie for example and see if it gonna recommend it for you or not using some threshold 



- Spark Ml use least squares ALS algorithm to learn latend factors the most difficult part to get readdy your data for such algorithm
- ALS is Matrix Factorisation Approch to implement a recommendation algoritm you decompose your large user / item into lower dimensional user factors and item factors 

# Creating a movie system recommender base en ALS 

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("rec").getOrCreate()
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [9]:
data = spark.read.csv("../Data/movielens_ratings.csv", header=True, inferSchema=True)

In [10]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [11]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [14]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [17]:
train_data , test_data = data.randomSplit([0.8,0.2])

In [18]:
asl = ALS(maxIter=5, regParam=0.01,userCol="userId",ratingCol="rating",itemCol="movieId")

In [20]:
model = asl.fit(train_data)

In [24]:
predictions = model.transform(test_data)

In [26]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|    13| 1.4069265|
|     31|   1.0|    19| 2.5479438|
|     31|   2.0|    25| 3.0240483|
|     31|   3.0|    14| 1.7381967|
|     85|   1.0|    13|0.41822946|
|     85|   2.0|    20| 1.6716951|
|     85|   1.0|     4|  2.906794|
|     65|   1.0|    16| 1.4387316|
|     65|   2.0|     3| 1.2897922|
|     53|   1.0|     6|  1.023263|
|     53|   2.0|    19| 1.1276473|
|     53|   5.0|     8| 3.9227157|
|     53|   1.0|    23| 0.6580797|
|     53|   1.0|     7|  3.879508|
|     78|   1.0|    28| 0.6351465|
|     78|   1.0|     2|0.54923487|
|     34|   1.0|    28| 1.5207725|
|     34|   1.0|    19| 1.2611029|
|     34|   1.0|    17| 2.0133095|
|     34|   1.0|    14|   3.17316|
+-------+------+------+----------+
only showing top 20 rows



# let Evaluate the models 

In [27]:
evaluator = RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")

In [29]:
rmse = evaluator.evaluate(predictions)

In [31]:
print("RMSE")
rmse

RMSE


1.995735419549549

In [32]:
# if we want to imporve this result the subject a little complecated we need to go in deep in mathmematical thing 

In [40]:
singler_user = test_data.filter(test_data["userId"] ==11).select(["movieId","userId"])

In [41]:
singler_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     10|    11|
|     13|    11|
|     16|    11|
|     18|    11|
|     22|    11|
|     48|    11|
|     50|    11|
|     51|    11|
|     67|    11|
|     76|    11|
+-------+------+



In [47]:
# we want to know if for his single use the prediction of his rating movies to see if we can extract
# recommendation 
recommendation = model.transform(singler_user)

In [48]:
recommendation.orderBy("prediction",ascending= False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     76|    11| 3.2415826|
|     18|    11| 3.2364187|
|     51|    11| 2.3619936|
|     13|    11| 2.3533688|
|     50|    11| 2.3429482|
|     10|    11| 2.0083745|
|      0|    11| 1.4527268|
|     22|    11| 1.3885461|
|     67|    11| 1.1777387|
|     48|    11| 0.4346084|
|     16|    11|0.36645466|
+-------+------+----------+

